# Setup 🏗️


In [27]:
import numpy as np
import torch 
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import StandardScaler
from random import randrange




In [28]:
viscos = 1/5200

# load RANS data created by rans.m (which can be downloaded)
# load DNS data
DNS_mean  = np.genfromtxt("LM_Channel_5200_mean_prof.dat",comments="%").transpose()
y_DNS     = DNS_mean[0]
yplus_DNS = DNS_mean[1]
u_DNS     = DNS_mean[2]
dudy_DNS  = np.gradient(u_DNS,y_DNS)

DNS_stress = np.genfromtxt("LM_Channel_5200_vel_fluc_prof.dat",comments="%").transpose()

uu_DNS = DNS_stress[2]
vv_DNS = DNS_stress[3]
ww_DNS = DNS_stress[4]
uv_DNS = DNS_stress[5]
uw_DNS = DNS_stress[6]
vw_DNS = DNS_stress[7]
k_DNS  = 0.5*(uu_DNS+vv_DNS+ww_DNS)

DNS_RSTE = np.genfromtxt("LM_Channel_5200_RSTE_k_prof.dat",comments="%")

eps_DNS = DNS_RSTE[:,7]/viscos # it is scaled with ustar**4/viscos

# fix wall
eps_DNS[0]=eps_DNS[1]

# load data from k-omega RANS
data  = np.loadtxt('y_u_k_om_uv_5200-RANS-code.txt').transpose()
y     = data[0]
u     = data[1]
k     = data[2]
om    = data[3]
diss1 = 0.09*k*om
ustar = (viscos*u[0]/y[0])**0.5
yplus = y*ustar/viscos

# dont train on, uu, vv, ww, uv, uw, vw 
# Maybe mixed terms are ok (just not uu,vv,ww)

#-----------------Data_manipulation--------------------

# Delete first value for all interesting data
uv_DNS    = np.delete(uv_DNS, 0)
vv_DNS    = np.delete(vv_DNS, 0)
ww_DNS    = np.delete(ww_DNS, 0)
uw_DNS    = np.delete(uw_DNS,0)
vw_DNS    = np.delete(vw_DNS,0)
k_DNS     = np.delete(k_DNS, 0)
eps_DNS   = np.delete(eps_DNS, 0)
dudy_DNS  = np.delete(dudy_DNS, 0)
yplus_DNS = np.delete(yplus_DNS,0)
uu_DNS    = np.delete(uu_DNS,0)
y_DNS     = np.delete(y_DNS,0)
u_DNS     = np.delete(u_DNS,0)

# Calculate ny_t and time-scale tau
viscous_t = k_DNS**2/eps_DNS 
tau       = viscous_t/abs(uv_DNS)

# Calculate c_1, c_2, & c_3 of the Non-linear Eddy Viscosity Model
# Array for storing c_1, c_2, & c_3
c_0 = -2*(ww_DNS/k_DNS - 2/3)/(tau**2*dudy_DNS**2)
c_2 = 2*((ww_DNS/k_DNS - 2/3) + (uu_DNS/k_DNS - 2/3))/(tau**2*dudy_DNS**2)

c = np.array([c_0,c_2])

dudy_squared_DNS = (dudy_DNS**2).reshape(-1,1)

#TODO ML using PyTorch to estimate c_1, c_2, & c_3


Setting up input and output as tensors 

In [55]:

def reshape_those_fuckers(*args):
    return [arg.reshape(-1,1) for arg in args]



X = StandardScaler().fit_transform(dudy_squared_DNS)

# transpose the target vector to make it a column vector  
y = c.transpose()

#tau, dudy, k, uu, vv, ww
test_var = np.concatenate((reshape_those_fuckers(tau,dudy_DNS,k_DNS,uu_DNS,vv_DNS,ww_DNS)),axis=1)

# split the feature matrix and target vector into training and validation sets
# test_size=0.2 means we reserve 20% of the data for validation
# random_state=42 is a fixed seed for the random number generator, ensuring reproducibility

random_state = randrange(100)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state= random_state)
X_train, X_val, test_var_train, test_var_val = train_test_split(X, test_var, test_size=0.2, random_state= random_state)

# convert the numpy arrays to PyTorch tensors with float32 data type
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)

# create PyTorch datasets and dataloaders for the training and validation sets
# a TensorDataset wraps the feature and target tensors into a single dataset
# a DataLoader loads the data in batches and shuffles the batches if shuffle=True
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [56]:
print(test_var_val)

[[4.25909270e+00 2.93730877e+00 1.11834401e+00 1.10485671e+00
  5.37564695e-01 5.94266619e-01]
 [3.78546697e-01 6.38877999e+01 4.53574815e+00 5.47252368e+00
  1.27249335e+00 2.32647928e+00]
 [3.35380532e-01 7.21907811e+01 4.58569441e+00 5.51862528e+00
  1.27524949e+00 2.37751406e+00]
 [6.80763871e-02 3.76136625e+03 3.82750555e+00 6.38968378e+00
  7.63908980e-02 1.18893643e+00]
 [2.64104883e+00 5.51658808e+00 1.83543233e+00 1.96924762e+00
  7.50777704e-01 9.50839337e-01]
 [2.89375650e-01 8.33822182e+01 4.64048664e+00 5.56472924e+00
  1.27729494e+00 2.43894911e+00]
 [1.53027714e+00 1.10008048e+01 3.08224024e+00 3.54955731e+00
  1.09178248e+00 1.52314068e+00]
 [3.12466645e+00 4.48536808e+00 1.48740851e+00 1.55726611e+00
  6.41766450e-01 7.75784460e-01]
 [1.83852988e+00 8.62364875e+00 2.69854045e+00 3.03717335e+00
  1.00337188e+00 1.35653566e+00]
 [8.04497616e-01 2.64799388e+01 4.01065387e+00 4.83354455e+00
  1.22924792e+00 1.95851527e+00]
 [7.40166366e-02 8.74094010e+02 5.81154986e+00 8.5

Let's set up a neural network:

In [32]:
class ThePredictionMachine(nn.Module):

    def __init__(self):
        
        super(ThePredictionMachine, self).__init__()

        self.input   = nn.Linear(1, 50)     
        self.hidden1 = nn.Linear(50, 25)    
        self.hidden2 = nn.Linear(25, 2)     

    def forward(self, x):
        x = nn.functional.relu(self.input(x))
        x = nn.functional.relu(self.hidden1(x))
        x = self.hidden2(x)
        return x


In [33]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss = 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()

    test_loss /= num_batches

    print(f"Avg loss: {test_loss:>8f} \n")

In [34]:
# Instantiate a neural network
neural_net = ThePredictionMachine()

# Set up hyperparameters
learning_rate = 1e-3
batch_size = 64
epochs = 1000

# Initialize the loss function
loss_fn = nn.MSELoss()

# Choose loss function, check out https://pytorch.org/docs/stable/optim.html for more info
# In this case we choose Stocastic Gradient Descent
optimizer = torch.optim.SGD(neural_net.parameters(), lr=learning_rate)


for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_loader, neural_net, loss_fn, optimizer)
    test_loop(val_loader, neural_net, loss_fn)
print("Done!")


Epoch 1
-------------------------------
Avg loss: 0.056475 

Epoch 2
-------------------------------
Avg loss: 0.046848 

Epoch 3
-------------------------------
Avg loss: 0.039010 

Epoch 4
-------------------------------
Avg loss: 0.032330 

Epoch 5
-------------------------------
Avg loss: 0.027104 

Epoch 6
-------------------------------
Avg loss: 0.022780 

Epoch 7
-------------------------------
Avg loss: 0.019191 

Epoch 8
-------------------------------
Avg loss: 0.016208 

Epoch 9
-------------------------------
Avg loss: 0.013724 

Epoch 10
-------------------------------
Avg loss: 0.011527 

Epoch 11
-------------------------------
Avg loss: 0.009802 

Epoch 12
-------------------------------
Avg loss: 0.008351 

Epoch 13
-------------------------------
Avg loss: 0.007129 

Epoch 14
-------------------------------
Avg loss: 0.006100 

Epoch 15
-------------------------------
Avg loss: 0.005231 

Epoch 16
-------------------------------
Avg loss: 0.004489 

Epoch 17
--------

In [35]:
preds = neural_net(X_val_tensor)

In [36]:
print(preds)


tensor([[ 8.1986e-04,  2.7755e-03],
        [ 8.7572e-04,  2.7906e-03],
        [ 8.7645e-04,  2.7908e-03],
        [ 8.7620e-04,  2.7907e-03],
        [ 8.7624e-04,  2.7908e-03],
        [ 8.7561e-04,  2.7906e-03],
        [ 8.7585e-04,  2.7906e-03],
        [ 8.7643e-04,  2.7908e-03],
        [ 8.4268e-04,  2.7817e-03],
        [ 8.6110e-04,  2.7867e-03],
        [ 8.7626e-04,  2.7907e-03],
        [ 7.9686e-04,  2.7693e-03],
        [ 8.7629e-04,  2.7907e-03],
        [ 8.7356e-04,  2.7900e-03],
        [-1.7907e-04,  2.5060e-03],
        [ 8.7465e-04,  2.7903e-03],
        [ 8.7525e-04,  2.7905e-03],
        [ 9.4712e-05,  2.5799e-03],
        [ 8.7500e-04,  2.7904e-03],
        [ 8.7514e-04,  2.7905e-03],
        [ 8.7645e-04,  2.7908e-03],
        [ 8.6227e-04,  2.7870e-03],
        [ 8.7634e-04,  2.7908e-03],
        [ 6.2423e-04,  2.7227e-03],
        [ 8.7494e-04,  2.7904e-03],
        [ 8.4456e-04,  2.7822e-03],
        [ 8.7540e-04,  2.7905e-03],
        [ 4.6063e-02, -8.789

In [37]:
print(y_val_tensor)

tensor([[6.2119e-04, 1.3829e-03],
        [1.4318e-03, 2.3836e-03],
        [1.9555e-03, 2.0980e-03],
        [1.7245e-03, 2.3779e-03],
        [1.7698e-03, 2.3615e-03],
        [1.4045e-03, 2.4021e-03],
        [1.4917e-03, 2.3954e-03],
        [1.9241e-03, 2.1602e-03],
        [7.0555e-04, 1.4818e-03],
        [8.7382e-04, 1.7072e-03],
        [1.7842e-03, 2.3530e-03],
        [5.7670e-04, 1.3422e-03],
        [1.8017e-03, 2.3353e-03],
        [1.2754e-03, 2.2986e-03],
        [3.9698e-04, 1.4340e-03],
        [1.3512e-03, 2.4236e-03],
        [1.3848e-03, 2.4377e-03],
        [4.1102e-04, 1.4597e-03],
        [1.3725e-03, 2.4423e-03],
        [1.3786e-03, 2.4381e-03],
        [1.9749e-03, 2.1433e-03],
        [8.9307e-04, 1.7338e-03],
        [1.8744e-03, 2.2639e-03],
        [4.7651e-04, 1.3561e-03],
        [1.3672e-03, 2.4374e-03],
        [7.1624e-04, 1.4953e-03],
        [1.3986e-03, 2.4428e-03],
        [2.4052e-05, 4.4559e-05],
        [1.4247e-03, 2.3812e-03],
        [8.048